In [12]:
import json
import sys
from dotenv import load_dotenv, find_dotenv
import os

sys.path.append('../../aisuite')

# Load from .env file if available
load_dotenv(find_dotenv())

True

### Define the function

In [13]:
def will_it_rain(location: str, time_of_day: str):
    """Check if it will rain in a location at a given time today.

    Args:
        location (str): Name of the city
        time_of_day (str): Time of the day in HH:MM format.
    """
    return "YES"

---

### Using OpenAI

### JSON spec for the function

In [14]:
tools = [{
    "type": "function",
    "function": {
        "name": "will_it_rain",
        "description": "Check if it will rain in a location at a given time today",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "Name of the city"
                },
                "time_of_day": {
                    "type": "string",
                    "description": "Time of the day in HH:MM format."
                }
            },
            "required": ["location", "time_of_day"]
        }
    }
}]

### Send user request to LLM

In [15]:
from openai import OpenAI

client = OpenAI()
messages = [{
    "role": "user",
    "content": "I live in San Francisco. Can you check for weather "
               "and plan an outdoor picnic for me at 2pm?"
}]
response = client.chat.completions.create(
    model="gpt-4o", messages=messages, tools=tools
)

### Process tool call response, Execute the tool & call the model again

In [16]:
response2 = None
if response.choices[0].message.tool_calls:
    tool_call = response.choices[0].message.tool_calls[0]
    args = json.loads(tool_call.function.arguments)

    result = will_it_rain(args["location"], args["time_of_day"])
    messages.append(response.choices[0].message)
    messages.append({
        "role": "tool", "tool_call_id": tool_call.id, "content": str(result)
    })

    response2 = client.chat.completions.create(
        model="gpt-4o", messages=messages, tools=tools)
    print(response2.choices[0].message.content)

It looks like it will rain in San Francisco at 2 PM today. It's probably not the best day for an outdoor picnic. Would you like to consider indoor activities or reschedule the picnic for another day?


### Optionally, continue the conversation

In [17]:
messages.append(response2.choices[0].message)
messages.append({
    "role": "user", "content": "Schedule it despite the rain"
})
response = client.chat.completions.create(
    model="gpt-4o", messages=messages)
print(response.choices[0].message.content)

If you're set on having the picnic despite the rain, here are a few suggestions to make it enjoyable:

1. **Location:** Consider a location with some shelter, like a gazebo in a park, or find a picnic spot with large trees for some natural cover. Alternatively, you might want to think about an indoor space with large windows where you can enjoy the view without getting wet.

2. **Packing Essentials:**
   - Bring waterproof blankets or tarps to sit on.
   - Pack an umbrella or a rain poncho to stay dry.
   - Use waterproof containers for your food to keep everything dry.
   - Bring extra towels and a change of clothes.

3. **Food Ideas:** Opt for warm, hearty meals that are comforting on a rainy day, such as soup in a thermos or grilled sandwiches.

4. **Activities:** Plan indoor-friendly games or activities you can enjoy sheltered from the rain, like card games or board games.

5. **Safety First:** Check the weather forecast regularly in case of any severe weather warnings.

Enjoy your

---

### Using aisuite

### Call the model with tools. Tool call is handled internally.

In [18]:
from aisuite import Client

client = Client()
messages = [{
    "role": "user",
    "content": "I live in San Francisco. Can you check for weather "
               "and plan an outdoor picnic for me at 2pm?"
}]
response = client.chat.completions.create(
    model="openai:gpt-4o", messages=messages, tools=[will_it_rain], max_turns=2
)
print(response.choices[0].message.content)

It is expected to rain in San Francisco at 2 PM today, so it might not be the best time for an outdoor picnic. You might want to consider indoor alternatives or plan for another day when the weather is more favorable for outdoor activities. If you have any other plans in mind or need further assistance, feel free to ask!


### Optionally, continue the conversation

In [19]:
messages.extend(response.choices[0].intermediate_messages)
messages.append({
    "role": "user", "content": "Schedule it despite the rain"
})
response = client.chat.completions.create(
    model="openai:gpt-4o", messages=messages)
print(response.choices[0].message.content)

Sure! If you'd like to go ahead and plan an outdoor picnic despite the rain, here are a few tips to help you enjoy your experience:

1. **Choose a Covered Location**: Try to find a spot with a shelter, like a gazebo or a pavilion, in one of San Francisco's parks. These can provide good protection from the rain.

2. **Prepare for Wet Weather**:
   - Bring waterproof blankets or tarps to sit on.
   - Pack umbrellas and raincoats to stay dry.
   - Use waterproof containers and bags to keep your picnic items protected.

3. **Select Comforting Foods**: Warm beverages in thermoses and foods that are enjoyable when slightly cooler, such as sandwiches, can be comforting on a rainy day.

4. **Stay Safe**: Be cautious of slippery surfaces and keep an eye on kids or pets if they are joining you.

5. **Enjoy the Atmosphere**: Rain can create a cozy and calming environment. Embrace the natural sounds and sights of the rain.

6. **Plan Indoor Activities**: Bring board games or books in case the rain

### Using, other providers

In [20]:
messages = [{
    "role": "user",
    "content": "I live in San Francisco. Can you check for weather "
               "and plan an outdoor picnic for me at 2pm?"
}]
response = client.chat.completions.create(
    model="anthropic:claude-3-5-sonnet-20240620", messages=messages, tools=[will_it_rain], max_turns=2
)
print(response.choices[0].message.content)

I apologize, but it looks like it will be raining in San Francisco at 2:00 PM today. This might not be the best conditions for an outdoor picnic. However, I can suggest a few alternatives:

1. Reschedule: We could check the weather for a different time today or another day this week. Would you like me to check a different time or date?

2. Indoor picnic: You could have an indoor picnic at home or find a covered area in a park with picnic shelters.

3. Rain-ready picnic: If you're up for an adventure, you could prepare for a rainy day picnic with appropriate gear like umbrellas, waterproof blankets, and covered food containers.

4. Alternative activity: We could look into indoor activities or attractions in San Francisco that you might enjoy instead.

What would you prefer to do? If you'd like to try for a different time or date, please let me know, and I'll be happy to check the weather again.


In [ ]:
response = client.chat.completions.create(
    model="groq:llama3-8b-8192", messages=messages, tools=[will_it_rain], max_turns=2)